In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import patches
from matplotlib.patches import Ellipse
from decimal import Decimal

In [ ]:
plt.rcParams['font.size'] = 7

In [ ]:
target = '~/Downloads/MSName_RandChunk0_DynSpecs_1556208112/TARGET/1556208112_14:29:42.950_-62:40:46.200.fits'
target_w = '~/Downloads/MSName_RandChunk0_DynSpecs_1556208112/TARGET_W/1556208112_14:29:42.950_-62:40:46.200.W.fits'
target_w2 = '~/Downloads/MSName_RandChunk0_DynSpecs_1556208112/TARGET_W/1556208112_14:29:42.950_-62:40:46.200.W2.fits'

target_hdu = fits.open(target)[0]
target_w_hdu = fits.open(target_w)[0]
target_w2_hdu = fits.open(target_w2)[0]

data = target_hdu.data
weights = target_w_hdu.data
weights2 = target_w2_hdu.data

header = target_hdu.header

In [ ]:
# Loop over the items and print each one
for keyword in header:
#     print(f'{keyword} = {header[keyword]}')
    frq_min = header['FRQ-MIN'] * 1e-6
    frq_max = header['FRQ-MAX'] * 1e-6
    tmin = 0
    tmax = data.shape[2] * header['CDELT1']

In [ ]:
weights_data = np.where(weights==0,1, weights)
weights_data2 = np.where(weights2==0,1, weights2)

sigma_inv = np.sqrt(np.power(weights_data, 2) / weights_data2)

for i in range(data.shape[0]):
    #data[i, :, :] = data[i, :, :] * norm_w * np.cbrt(data[i, :, :])
    data[i, :, :] = data[i, :, :] * sigma_inv

print('--- Data has shape ---')
print('(stokes, freq, time)')
print(data.shape)

In [ ]:
stokes_idx = 0
d = data[stokes_idx, :, :]
extent=[tmin, tmax, frq_min, frq_max]
plt.imshow(d, origin='lower', cmap='viridis', aspect='auto', vmin=np.min(d), vmax=np.max(d), extent=extent)
plt.title('Dynamic Spectrum of total intensity (Stokes I)')
plt.xlabel('time (s)')
plt.ylabel(r'$\nu$ (MHz)')
cbar_t = plt.colorbar(pad=.07)
cbar_t.set_label(r'$mJy/B$', size=7)

# Stokes I, Q, U, V Plotted in order below
```python
I = data[0, :, :]
Q = data[1, :, :]
U = data[2, :, :]
V = data[3, :, :]
```

For now, we slice the time (t0, t1):
```sh
6325, 6345
12953, 12993
19803, 19843
```

In [ ]:
def get_spectral_features(s, t0, t1):
    data_sl = data[s, :, t0:t1]
    t = []
    if not np.max(data_sl) == 0:
        t.append(t0)
        print("Flaring from {} to {} seconds".format(t0, t1))
    return

In [ ]:
def plot_stokes(stokes_symbol, stokes_idx, t0, t1):
    """Plot dynamic spectra for given data array and Stokes param
    
    Parameters
    ----------
    stokes_symbol: str
        Stokes parameter of interest mapped ('I', 'Q', 'U', 'V')
    stokes_idx: int
        Stokes parameter of interest mapped ('I', 'Q', 'U', 'V')->(0, 1, 2, 3)
    t0: int
        `start_time` to zoom-in on
    t1: int
        `end_time to` zoom-in on
    """
    data_sl = data[stokes_idx, :, t0:t1]
    print("--- Stats for Stokes {} ---".format(stokes_symbol))
    print("----Slicing throug time axis from {} to {} seconds after start of observation".format(t0, t1))
    print("Min: ", np.min(data_sl))
    print("Max: ", np.max(data_sl))
    print("Noise: ", np.std(data_sl))
    print("Mean: ", np.mean(data_sl))
    #print(np.argmax(d[stokes_idx, :, :]))

    # Plot
    extent = [t0, t1, frq_min, frq_max]
    plt.title(stokes_symbol)
    plt.imshow(data_sl, origin='lower', cmap='viridis', aspect='auto', vmin=np.min(data_sl), vmax=np.max(data_sl), extent=extent)
    plt.xlabel('time (s)')
    plt.ylabel(r'$\nu$ (MHz)')
    cbar = plt.colorbar(pad=.07)
    cbar.set_label(r'$mJy/B$', size=7)

In [ ]:
for i in range(0, data.shape[2], 20):
    t0, t1 = i, i+20
    get_spectral_features(0, t0, t1)

Flaring events every 35 minutes that repeat every 20 seconds

In [ ]:
(19820-17440)/60

In [ ]:
plot_stokes('Stokes I', 0, 0, 40)

In [ ]:
# plot_stokes(data, 'Stokes I', 0, 12953, 12993)

In [ ]:
# plot_stokes(data, 'Stokes I', 0, 19803, 19843)

In [ ]:
# plot_stokes(data, 'Stokes I', 0, 19803, 19843)

In [ ]:
# plot_stokes(data, 'Stokes Q', 1, 19803, 19843)

In [ ]:
# plot_stokes(data, 'Stokes U', 2, 19803, 19843)

In [ ]:
# plot_stokes(data, 'Stokes V', 3, 19803, 19843)

### During Fast Imaging meeting
Oleg advised I try this
``` sh
pip install owlcat
plot-evelation-tracks.py <MS>
```
This will help us determine if the features we see were during the duty cycle of the target source

![](lst-elev.png)